In [120]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cadd_io as cdio
import cadd_main as cdmain
import cadd_plot as cdplot
import mdutilities_io as mduio
import mymath as Mmath
import copy
import my_plot as myplot
import cadddatadump as cddump
import cadd_mesh2 as cdmesh
import cadd_crack_mesh as cdcmesh
import cadd
import itertools
import newpotential as newpot
import tabularpotential as tabpot
import crystallography_oo as croo
import lineplot_oo3 as lpoo
%matplotlib qt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preliminaries

## Directories

In [121]:
maindir = '../Tests/CADD_K_Test/'
uidir = maindir + 'User Inputs/'
fidir = maindir + 'Fortran Inputs/'
dumpdir = maindir + 'Dump Files/'
restartdir = maindir + 'Restart Files/'
simname = 'cadd_k_test'
simtype = 'cadd'

In [122]:
size = 'medium'
if size == 'small':
    ferings = 5
elif size == 'medium':
    ferings = 14
elif size == 'large':
    ferings = 24
simname += '_' + size

## Unit System

In [123]:
# LJ
r0 = 1.0
Alr0 = 125e-12
Almass = (27)/(6.02e23)
# Alcohenergy = 5.4e-19 # cohesive energy/atom
Almu = 28e12
Altime = np.sqrt(Almass/(Almu*Alr0))
myr0 = r0
mymass = 1.0
mymu = 13.4
# mycohenergy = 6.0
mytime = np.sqrt(mymass/(mymu*myr0))
lengthfac = myr0/Alr0
timefac = mytime/Altime
massfac = mymass/Almass
stressfac = massfac/(lengthfac*timefac**2)

# Create Inputs

## Nodes/Elements

### Create Mesh

In [124]:
padx, pady = 5, 5
lx, ly = 42, 45 # gives roughly 2500 atoms (including pad, interface)

# initial mesh
crackmesh = cdcmesh.CrackMesh(ferings,lx,ly,padx,pady,r0)

# fix outer nodes for K-test
outernodes = crackmesh.outerbox.all_nodes
crackmesh.mesh.nodes.set_node_bc(outernodes,3)

# modify mesh to separate crack faces
crackmesh.modify_mesh()

mesh = crackmesh.mesh
meshnodes = mesh.nodes

### Plot

In [125]:
mesh.write_dump_all('test.7.dump')
sim = cdmain.Simulation('cadd_nodisl','test',readinput=False)
fig = sim.plot_dump_from_file('test.7.dump',fignum=2)

### Nodes

In [126]:
n = meshnodes.xy.shape[0]
z = np.zeros((n,))
posn = np.column_stack((meshnodes.xy,z)) # z-coordinate = 0
nodes = cdmain.Nodes(posn=posn,types=meshnodes.types)

### FE Elements

In [127]:
feelements = cdmain.FEElement(elname='CPE3',mnum=1,
                              connect=mesh.elements_dump) # elements_dump fixes off by 1 indexing issue

## Potential

In [128]:
ductilityindex = 6 # scale of 0 to 6 in 2D
potentialname = 'ductilenewnorm'

In [129]:
tunablepotential = newpot.gen_std_potential(ductilityindex=ductilityindex,dimensions=2,r0=1)

In [130]:
rminsmall, rmin, rmax = 0.01, 0.1, 2.0
npoints = 3000
rvec = np.linspace(rmin,rmax,npoints)
rvec = np.insert(rvec,0,rminsmall)
pottable = tunablepotential.get_energy_force_table(rvec)

In [131]:
tunablepotential.write_file_lammps(potentialname,np.linspace(0.1,3.0,3000),extend=True)

In [132]:
# Energy plot
# lpoo.my_quick_plot([pottable[:,[0,1]]],axisbounds=[0.5,2,-1.1,3])

In [133]:
rnnn = np.sqrt(3)*r0 # 2nd nearest neighbor
potential = cdmain.Potential(pname=potentialname,forcecutoff=rnnn,pottable=pottable)

## Material

### Elastic constants

In [134]:
elasticdict = tunablepotential.elasticdict('hex')
tunablemat = croo.Hex.from_potential(tunablepotential)
elconst = tunablemat.voigt_plane_strain_stiffness

In [135]:
elconst

array([[ 40.09879362,  13.36626454,   0.        ],
       [ 13.36626454,  40.09879362,   0.        ],
       [  0.        ,   0.        ,  13.36626454]])

### Dislocation stuff

In [136]:
lannih = 6*r0
aluminumdislvmax = 1000
mydislvmax = aluminumdislvmax*(lengthfac/timefac)
aluminumdisldrag = 1e-4
mydisldrag = aluminumdisldrag*(stressfac*timefac)

In [137]:
material = cdmain.Material(burgers=r0,disldrag=mydisldrag,dislvmax=mydislvmax,elconst=elconst,
                           lannih=lannih,lattice='hex',mass=1.0,mname='inter1newnorm',rho=tunablepotential.rho)

## Misc

In [138]:
misc = cdmain.Misc(increments=100,timestep=0.02,iscrackproblem=True,potstyle='table')

## Interactions

In [139]:
interactions = cdmain.Interactions(table=np.array([[1,1,1]]))

## Neighbors

In [140]:
neighbors = cdmain.Neighbors(checkdisp=True,every=0,delay=5,images=0,Lz=1.0,skin=0.3)

## Damping

In [141]:
gamma = 0.1 # may need to fiddle with this
damping = cdmain.Damping(flag=True,gname='all',gamma=gamma)

## DD Stuff

In [142]:
# no dislocations, sources, or obstacles
escapeddisl = cdmain.EscapedDislocations()
ghostdisl = cdmain.GhostDislocations()
disl = cdmain.Dislocations()
sources = cdmain.Sources()
obstacles = cdmain.Obstacles()

### Slip Systems

In [143]:
# just a few slip planes accessible from atomistic region (i.e., close to the origin)
# fudges are to ensure full coverage
nslipsys = 3
xmin, xmax, ymin, ymax = crackmesh.interfacebox.bounds
fudge, fudge2 = 1.1, 1.5
planespacing = r0*np.sqrt(3)/2
maxdist = fudge*np.sqrt((xmax-xmin)**2 + (ymax-ymin)**2)
nplanes = fudge2*int(round(maxdist/planespacing))

In [144]:
# recall that origin is at very edge of slip system --- slip planes should be built up in only one direction
theta = np.empty((nslipsys,))
origin = np.empty((nslipsys,2))
nslipplanes = nplanes*np.ones((nslipsys,))
spacing = planespacing*np.ones((nslipsys,))

#### System 1 (theta = 0)

In [145]:
theta[0] = 0.0
origin[0,:] = np.array([0,fudge*ymin])

#### System 2 (theta = 60)

In [146]:
theta[1] = np.pi/3
origin[1,:] = np.array([fudge*xmax,fudge*ymin])

#### System 3 (theta = -60)

In [147]:
theta[2] = -np.pi/3
origin[2,:] = np.array([fudge*xmin,fudge*ymin])

In [148]:
slipsys = cdmain.SlipSystem(theta=theta,origin=origin,nslipplanes=nslipplanes,space=spacing)

## Compute

In [149]:
neighborsnum = 6
centro = cdmain.ComputeData(params=np.array([neighborsnum]),active=True,gname='all')
compute = cdmain.Compute(centro=centro)

## Detection

### Detection Annulus

In [150]:
bandtype = 'rect_annulus'

# get center of interface box
ptsall = np.array([line.startpoint for line in crackmesh.interfacebox.lines])
xcenter, ycenter = np.mean(ptsall,axis=0)

# move outer part of detection annulus slightly inward from interface box
fudge = 3.0
_, xmax, _, ymax = crackmesh.interfacebox.bounds
halfLxouter = xmax - xcenter - fudge
halfLyouter = ymax - ycenter - fudge

In [151]:
xthickness, ythickness = 3, 3
halfLxinner = halfLxouter - xthickness
halfLyinner = halfLyouter - ythickness

In [152]:
params = np.array([xcenter,ycenter,halfLxinner,halfLxouter,halfLyinner,halfLyouter])

In [153]:
params

array([  0.        ,  -0.11473671,  15.        ,  18.        ,
        16.5166605 ,  19.5166605 ])

### Crack Edges

In [154]:
fac = 10.0 # extend crack face by -fac from interfacenode
crackatomedges = crackmesh.crack_atom_edges(fac)

### Pass Distances

In [155]:
maxdisttointerface = 10.0 # from inner part of detection band to interface
passdistancectoa = 3.0
passdistanceatoc = 8.0

### Other

In [156]:
# may have to fiddle with these
dampgamma = 0.1
detectiondamp = cdmain.Damping.temp_damping(gamma=dampgamma)
mdnincrements = 500
mdtimestep = 0.02
mnumfe = 1

### All

In [157]:
detection = cdmain.Detection(bandtype=bandtype,damp=detectiondamp,impermissibleedges=crackatomedges,
                             mdnincrements=mdnincrements,mdtimestep=mdtimestep,mnumfe=mnumfe,
                             params=params,maxdisttointerface=maxdisttointerface,
                             passdistanceatoc=passdistanceatoc,passdistancectoa=passdistancectoa)

## Everything

In [158]:
cadddata = cdmain.CADDData(simtype,nfematerials=1,nodes=nodes,materials=[material],misc=misc,groups=None,compute=compute,
                               potentials=[potential],interactions=interactions,neighbors=neighbors,damping=damping,
                               feelements=[feelements],escapeddisl=[escapeddisl],ghostdisl=[ghostdisl],disl=[disl],
                               sources=[sources],obstacles=[obstacles],slipsys=[slipsys],detection=detection)
caddsim = cdmain.Simulation(simtype,simname,uidir,fidir,dumpdir,restartdir,data=cadddata)

## Write

In [159]:
caddsim.write_fortran_all()

# Restart Simulation

# Misc

# Dump Files

In [163]:
fig = caddsim.plot_dump_from_increment(0,fignum=2);

In [171]:
fig = caddsim.plot_dump_from_increment(6,style='centro',fignum=2,axisbounds=[-35,35,-35,35]);

In [174]:
fig = caddsim.plot_dump_from_increment(9,style='centro',fignum=3,axisbounds=[-35,35,-35,35]);

In [ ]:
fig = caddsim.plot_dump_from_increment(41,fignum=2);

In [ ]:
mu = tunablepotential.elasticdict('hex')['66']
cR = 0.92*np.sqrt(mu/tunablepotential.rho)
cR